<a href="https://colab.research.google.com/github/ghommidhWassim/GNN-variants/blob/main/test_gcn_variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!python -c "import torch; print(torch.__version__)"


2.6.0+cu124


In [3]:
!python -c "import torch; print(torch.version.cuda)"



12.4


In [4]:
pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [5]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html



Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00


In [6]:
# Standard libraries
import numpy as np
from scipy import sparse
import seaborn as sns
import pandas as pd
import time

# Plotting libraries
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import cm
from IPython.display import Javascript  # Restrict height of output cell.

# PyTorch
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch.nn as nn
# import pyg_lib
import torch_sparse
from sklearn.metrics import f1_score

# PyTorch geometric
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid, Amazon
from torch_geometric.loader import ClusterData, ClusterLoader
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
from torch_geometric import seed_everything
from torch_geometric.nn.models import GraphSAGE
from torch_geometric.transforms import NormalizeFeatures, RandomNodeSplit
import torch_geometric.transforms as T
import json


**STANDARD GCN pubmed**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())
num_features = dataset.num_features
num_classes = dataset.num_classes
data = dataset[0].to(device)  # Get the first graph object.
data

Using device: cuda


Processing...
Done!


Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [ ]:
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')

Number of nodes:          19717
Number of edges:          88648
Average node degree:      4.50
Number of training nodes: 60
Training node label rate: 0.003
Has isolated nodes:       False
Has self-loops:           False
Is undirected:            True


In [20]:
def clean_gpu_memory():
    """Cleans GPU memory without fully resetting the CUDA context"""
    import gc
    gc.collect()  # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # PyTorch cache
        torch.cuda.reset_peak_memory_stats()  # Reset tracking
        print(f"Memory after cleanup: {torch.cuda.memory_allocated()/1024**2:.2f} MB")


In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=-1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 64, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

print(model)

GCN(
  (conv1): GCNConv(500, 64)
  (conv2): GCNConv(64, 3)
)


In [ ]:
def train(data, mask):
  model.train()
  optimizer.zero_grad()  # Clear gradients.
  out = model(data.x, data.edge_index)  # Perform a single forward pass.
  loss = criterion(out[mask], data.y[mask])  # Compute the loss solely based on the training nodes.
  loss.backward()  # Derive gradients.
  optimizer.step()  # Update parameters based on gradients.
  return loss

def test(data, mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    true = data.y[mask].cpu()
    pred = pred[mask].cpu()
    acc = (pred == true).sum().item() / mask.sum().item()
    micro_f1 = f1_score(true, pred, average='micro')
    return acc, micro_f1


In [ ]:

#display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

res=[]
training=[]
memory=[]
for i in range(10):
  clean_gpu_memory()
  start_time = time.time()
  for epoch in range(1, 101):
      loss = train(data, data.train_mask)
      train_acc, train_f1 = test(data, data.train_mask)
      val_acc, val_f1 = test(data, data.val_mask)
      print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')
  end_time = time.time()
  test_acc,f1_micro = test(data, data.test_mask)
  res.append(test_acc)
  training.append(end_time - start_time)
  memory.append(torch.cuda.max_memory_allocated()/1024**2)
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
print(f"Time taken: {end_time - start_time:.2f} seconds")

Memory after cleanup: 39.69 MB
Epoch: 001, Train Acc: 0.6500, Train F1: 0.6500, Val Acc: 0.5560, Val F1: 0.5560
Epoch: 002, Train Acc: 0.8833, Train F1: 0.8833, Val Acc: 0.7140, Val F1: 0.7140
Epoch: 003, Train Acc: 0.8833, Train F1: 0.8833, Val Acc: 0.7240, Val F1: 0.7240
Epoch: 004, Train Acc: 0.8833, Train F1: 0.8833, Val Acc: 0.7260, Val F1: 0.7260
Epoch: 005, Train Acc: 0.9000, Train F1: 0.9000, Val Acc: 0.7280, Val F1: 0.7280
Epoch: 006, Train Acc: 0.9167, Train F1: 0.9167, Val Acc: 0.7320, Val F1: 0.7320
Epoch: 007, Train Acc: 0.9167, Train F1: 0.9167, Val Acc: 0.7320, Val F1: 0.7320
Epoch: 008, Train Acc: 0.9167, Train F1: 0.9167, Val Acc: 0.7340, Val F1: 0.7340
Epoch: 009, Train Acc: 0.9167, Train F1: 0.9167, Val Acc: 0.7440, Val F1: 0.7440
Epoch: 010, Train Acc: 0.9167, Train F1: 0.9167, Val Acc: 0.7440, Val F1: 0.7440
Epoch: 011, Train Acc: 0.9167, Train F1: 0.9167, Val Acc: 0.7440, Val F1: 0.7440
Epoch: 012, Train Acc: 0.9333, Train F1: 0.9333, Val Acc: 0.7460, Val F1: 0.74

In [ ]:
std_acc = np.std(res)
mean_test_acc = np.mean(res)
print(f"Test Accuracy: {mean_test_acc:.4f} ± {std_acc:.4f}")
std_training = np.std(training)
mean_training = np.mean(training)
print(f"training time: {mean_training:.4f} ± {std_training:.4f}")
std_memory = np.std(memory)
mean_memory = np.mean(memory)
print(f"memory used: {mean_memory:.4f} ± {std_memory:.4f}")

Test Accuracy: 0.7577 ± 0.0017
training time: 1.7659 ± 0.3984
memory used: 121.8216 ± 2.7152


In [ ]:
out = model(data.x, data.edge_index)

In [ ]:
def gcn_memory_usage(num_layers: int, num_nodes: int, feat_dim: int, use_bytes: bool = True):

    # Each embedding matrix is size |V| × K floats, there are L such layers
    num_emb_floats = num_layers * num_nodes * feat_dim
    # Each weight matrix is K × K floats per layer
    num_weight_floats = num_layers * feat_dim * feat_dim

    if use_bytes:
        bytes_per_float = 4  # assuming float32
        embeddings_bytes = num_emb_floats * bytes_per_float
        weights_bytes = num_weight_floats * bytes_per_float
        total_bytes = embeddings_bytes + weights_bytes
        return embeddings_bytes, weights_bytes, total_bytes
    else:
        return num_emb_floats, num_weight_floats, num_emb_floats + num_weight_floats


e, w, t = gcn_memory_usage(2, data.num_nodes, out.shape[1])
print(f"Embeddings: {e/1e6:.1f} MB, Weights: {w*4/1e6:.1f} MB, Total: {t/1e6:.1f} MB")
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "GCN full-batch",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "embedding_storage":e,
    "Weight_Matrices":w,
    "Total_Memory":t
}

with open("GCN_full_batch_pubmed_results.json", "w") as f:
    json.dump(metrics, f)

Embeddings: 0.5 MB, Weights: 0.0 MB, Total: 0.5 MB


**Cora dataset**

In [ ]:
clean_gpu_memory()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
num_features = dataset.num_features
num_classes = dataset.num_classes
data = dataset[0].to(device)  # Get the first graph object.
data
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 64, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

res=[]
training=[]
memory=[]
for i in range(10):
  clean_gpu_memory()
  start_time = time.time()
  for epoch in range(1, 101):
      loss = train(data, data.train_mask)
      train_acc, train_f1 = test(data, data.train_mask)
      val_acc, val_f1 = test(data, data.val_mask)
      print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')
  end_time = time.time()
  test_acc,f1_micro = test(data, data.test_mask)
  res.append(test_acc)
  training.append(end_time - start_time)
  memory.append(torch.cuda.max_memory_allocated()/1024**2)


Memory after cleanup: 56.31 MB
Using device: cuda


Processing...
Done!


Number of nodes:          2708
Number of edges:          10556
Average node degree:      3.90
Number of training nodes: 140
Training node label rate: 0.052
Has isolated nodes:       False
Has self-loops:           False
Is undirected:            True
Memory after cleanup: 71.65 MB
Epoch: 001, Train Acc: 0.6000, Train F1: 0.6000, Val Acc: 0.4540, Val F1: 0.4540
Epoch: 002, Train Acc: 0.8071, Train F1: 0.8071, Val Acc: 0.4700, Val F1: 0.4700
Epoch: 003, Train Acc: 0.9071, Train F1: 0.9071, Val Acc: 0.5900, Val F1: 0.5900
Epoch: 004, Train Acc: 0.9500, Train F1: 0.9500, Val Acc: 0.6400, Val F1: 0.6400
Epoch: 005, Train Acc: 0.9571, Train F1: 0.9571, Val Acc: 0.6580, Val F1: 0.6580
Epoch: 006, Train Acc: 0.9571, Train F1: 0.9571, Val Acc: 0.6900, Val F1: 0.6900
Epoch: 007, Train Acc: 0.9500, Train F1: 0.9500, Val Acc: 0.6920, Val F1: 0.6920
Epoch: 008, Train Acc: 0.9643, Train F1: 0.9643, Val Acc: 0.6900, Val F1: 0.6900
Epoch: 009, Train Acc: 0.9571, Train F1: 0.9571, Val Acc: 0.6880, Val 

In [ ]:
std_acc = np.std(res)
mean_test_acc = np.mean(res)
print(f"Test Accuracy: {mean_test_acc:.4f} ± {std_acc:.4f}")
std_training = np.std(training)
mean_training = np.mean(training)
print(f"training time: {mean_training:.4f} ± {std_training:.4f}")
std_memory = np.std(memory)
mean_memory = np.mean(memory)
print(f"memory used: {mean_memory:.4f} ± {std_memory:.4f}")

Test Accuracy: 0.7817 ± 0.0034
training time: 1.4787 ± 0.2906
memory used: 80.4691 ± 0.5184


**Citeseer dataset**

In [ ]:
clean_gpu_memory()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
dataset = Planetoid(root='data/Planetoid', name='CiteSeer', transform=NormalizeFeatures())
num_features = dataset.num_features
num_classes = dataset.num_classes
data = dataset[0].to(device)  # Get the first graph object.
data
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 64, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

res=[]
training=[]
memory=[]

for i in range(10):
  clean_gpu_memory()
  start_time = time.time()

  for epoch in range(1, 101):
      loss = train(data, data.train_mask)
      train_acc, train_f1 = test(data, data.train_mask)
      val_acc, val_f1 = test(data, data.val_mask)
      print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')


  end_time = time.time()
  test_acc,f1_micro = test(data, data.test_mask)
  res.append(test_acc)
  training.append(end_time - start_time)
  memory.append(torch.cuda.max_memory_allocated()/1024**2)


std_acc = np.std(res)
mean_test_acc = np.mean(res)
print(f"Test Accuracy: {mean_test_acc:.4f} ± {std_acc:.4f}")
std_training = np.std(training)
mean_training = np.mean(training)
print(f"training time: {mean_training:.4f} ± {std_training:.4f}")
std_memory = np.std(memory)
mean_memory = np.mean(memory)
print(f"memory used: {mean_memory:.4f} ± {std_memory:.4f}")

Memory after cleanup: 72.22 MB
Using device: cuda


Processing...
Done!


Number of nodes:          3327
Number of edges:          9104
Average node degree:      2.74
Number of training nodes: 120
Training node label rate: 0.036
Has isolated nodes:       True
Has self-loops:           False
Is undirected:            True
Memory after cleanup: 104.60 MB
Epoch: 001, Train Acc: 0.7083, Train F1: 0.7083, Val Acc: 0.4140, Val F1: 0.4140
Epoch: 002, Train Acc: 0.7167, Train F1: 0.7167, Val Acc: 0.3480, Val F1: 0.3480
Epoch: 003, Train Acc: 0.8000, Train F1: 0.8000, Val Acc: 0.3980, Val F1: 0.3980
Epoch: 004, Train Acc: 0.9167, Train F1: 0.9167, Val Acc: 0.4860, Val F1: 0.4860
Epoch: 005, Train Acc: 0.9333, Train F1: 0.9333, Val Acc: 0.5320, Val F1: 0.5320
Epoch: 006, Train Acc: 0.9500, Train F1: 0.9500, Val Acc: 0.5640, Val F1: 0.5640
Epoch: 007, Train Acc: 0.9417, Train F1: 0.9417, Val Acc: 0.6040, Val F1: 0.6040
Epoch: 008, Train Acc: 0.9583, Train F1: 0.9583, Val Acc: 0.6280, Val F1: 0.6280
Epoch: 009, Train Acc: 0.9583, Train F1: 0.9583, Val Acc: 0.6520, Val F

**Amazon dataset**

In [ ]:
clean_gpu_memory()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
dataset = Amazon(
        root='data/Amazon',
        name='Computers',
        transform=T.Compose([
        NormalizeFeatures(),          # feature‑wise ℓ₂ normalisation
        RandomNodeSplit(              # ⇦ add a split transform
                split='train_rest',       # 10% val, 10% test by default
                num_val=0.1,
                num_test=0.1,
                num_splits=1,
            )
        ])
    )
num_features = dataset.num_features
num_classes = dataset.num_classes
data = dataset[0].to(device)  # Get the first graph object.
data
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 64, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

res=[]
training=[]
memory=[]

for i in range (10):
  clean_gpu_memory()
  start_time = time.time()

  for epoch in range(1, 101):
      loss = train(data, data.train_mask)
      train_acc, train_f1 = test(data, data.train_mask)
      val_acc, val_f1 = test(data, data.val_mask)
      print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')


  end_time = time.time()
  test_acc,f1_micro = test(data, data.test_mask)
  res.append(test_acc)
  training.append(end_time - start_time)
  memory.append(torch.cuda.max_memory_allocated()/1024**2)


std_acc = np.std(res)
mean_test_acc = np.mean(res)
print(f"Test Accuracy: {mean_test_acc:.4f} ± {std_acc:.4f}")
std_training = np.std(training)
mean_training = np.mean(training)
print(f"training time: {mean_training:.4f} ± {std_training:.4f}")
std_memory = np.std(memory)
mean_memory = np.mean(memory)
print(f"memory used: {mean_memory:.4f} ± {std_memory:.4f}")

Memory after cleanup: 106.61 MB
Using device: cuda


Processing...
Done!


Number of nodes:          13752
Number of edges:          491722
Average node degree:      35.76
Number of training nodes: 11002
Training node label rate: 0.800
Has isolated nodes:       True
Has self-loops:           False
Is undirected:            True
Memory after cleanup: 107.84 MB
Epoch: 001, Train Acc: 0.3778, Train F1: 0.3778, Val Acc: 0.3578, Val F1: 0.3578
Epoch: 002, Train Acc: 0.3778, Train F1: 0.3778, Val Acc: 0.3578, Val F1: 0.3578
Epoch: 003, Train Acc: 0.3778, Train F1: 0.3778, Val Acc: 0.3578, Val F1: 0.3578
Epoch: 004, Train Acc: 0.3778, Train F1: 0.3778, Val Acc: 0.3578, Val F1: 0.3578
Epoch: 005, Train Acc: 0.3778, Train F1: 0.3778, Val Acc: 0.3578, Val F1: 0.3578
Epoch: 006, Train Acc: 0.3778, Train F1: 0.3778, Val Acc: 0.3578, Val F1: 0.3578
Epoch: 007, Train Acc: 0.3778, Train F1: 0.3778, Val Acc: 0.3578, Val F1: 0.3578
Epoch: 008, Train Acc: 0.3778, Train F1: 0.3778, Val Acc: 0.3578, Val F1: 0.3578
Epoch: 009, Train Acc: 0.3778, Train F1: 0.3778, Val Acc: 0.3578,

In [87]:
import os
import zipfile

# Make a folder for figures
os.makedirs("figures", exist_ok=True)

def plot_metric(df, title, ylabel, log_scale=False, rotation=45, figsize=(14, 7), filename=None):
    plt.figure(figsize=figsize)
    ax = sns.barplot(data=df, palette="husl")

    plt.title("\n".join(wrap(title, 60)), fontsize=14, pad=20)
    plt.ylabel(ylabel, fontsize=12)
    plt.xlabel('Model', fontsize=12)
    plt.xticks(rotation=rotation, ha='right', fontsize=10)
    plt.yticks(fontsize=10)

    if log_scale:
        plt.yscale('log')
        plt.ylabel(f'{ylabel} (log scale)', fontsize=12)

    for p in ax.patches:
        if not np.isnan(p.get_height()):
            value = p.get_height()
            fmt = "{:.1e}" if log_scale and value > 1000 else "{:.2f}"
            ax.annotate(fmt.format(value),
                       (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha='center', va='center',
                       xytext=(0, 5),
                       textcoords='offset points',
                       fontsize=8)

    plt.tight_layout()

    if filename:
        plt.savefig(f"figures/{filename}.png", dpi=300)

    plt.close()  # Close to avoid display in Colab

def plot_dataset_comparison(dataset):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle(f'Performance Comparison for {dataset}', fontsize=16, y=1.05)

    acc_values = [accuracy_data[model][datasets.index(dataset)] if dataset in datasets and len(accuracy_data[model]) > datasets.index(dataset) else None for model in models]
    time_values = [time_data[model][datasets.index(dataset)] if dataset in datasets and len(time_data[model]) > datasets.index(dataset) else None for model in models]
    mem_values = [memory_data[model][datasets.index(dataset)] if dataset in datasets and len(memory_data[model]) > datasets.index(dataset) else None for model in models]

    sns.barplot(x=models, y=acc_values, ax=ax1, palette="husl")
    ax1.set_title('Accuracy')
    ax1.set_ylim(0, 1)
    ax1.set_xticklabels(models, rotation=45, ha='right')
    for p in ax1.patches:
        if not np.isnan(p.get_height()):
            ax1.annotate(f"{p.get_height():.3f}",
                         (p.get_x() + p.get_width() / 2., p.get_height()),
                         ha='center', va='center',
                         xytext=(0, 5),
                         textcoords='offset points',
                         fontsize=8)

    sns.barplot(x=models, y=time_values, ax=ax2, palette="husl")
    ax2.set_title('Training Time (seconds)')
    ax2.set_yscale('log')
    ax2.set_xticklabels(models, rotation=45, ha='right')
    for p in ax2.patches:
        if not np.isnan(p.get_height()):
            ax2.annotate(f"{p.get_height():.2f}",
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center',
                        xytext=(0, 5),
                        textcoords='offset points',
                        fontsize=8)

    sns.barplot(x=models, y=mem_values, ax=ax3, palette="husl")
    ax3.set_title('Memory Usage (MB)')
    ax3.set_xticklabels(models, rotation=45, ha='right')
    for p in ax3.patches:
        if not np.isnan(p.get_height()):
            ax3.annotate(f"{p.get_height():.1f}",
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='center',
                        xytext=(0, 5),
                        textcoords='offset points',
                        fontsize=8)

    plt.tight_layout()
    plt.savefig(f"figures/performance_{dataset}.png", dpi=300)
    plt.close()

# Generate and save dataset-specific comparisons
for dataset in datasets:
    plot_dataset_comparison(dataset)

# Zip all saved figures
with zipfile.ZipFile("figures.zip", "w") as zipf:
    for root, _, files in os.walk("figures"):
        for file in files:
            zipf.write(os.path.join(root, file), file)

print("All figures saved and zipped as figures.zip")


/tmp/ipython-input-1519956380.py:47: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=models, y=acc_values, ax=ax1, palette="husl")
/tmp/ipython-input-1519956380.py:50: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax1.set_xticklabels(models, rotation=45, ha='right')
/tmp/ipython-input-1519956380.py:60: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=models, y=time_values, ax=ax2, palette="husl")
/tmp/ipython-input-1519956380.py:63: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(models, rotation=45, ha='rig

All figures saved and zipped as figures.zip


In [88]:
from google.colab import files
files.download("figures.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!python pytorch_ladies.py --cuda 0 --dataset cora

cora ladies
/content/LADIES/pytorch_ladies.py:190: DeprecationWarning: Please import `lil_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.lil` namespace is deprecated and will be removed in SciPy 2.0.0.
  if type(feat_data) == scipy.sparse.lil.lil_matrix:
----------
Epoch: 0 (0.8s) Train Loss: 1.83    Valid Loss: 1.75 Valid F1: 0.674
Epoch: 1 (1.1s) Train Loss: 0.70    Valid Loss: 0.94 Valid F1: 0.714
Epoch: 2 (1.4s) Train Loss: 0.10    Valid Loss: 1.17 Valid F1: 0.684
Epoch: 3 (1.7s) Train Loss: 0.06    Valid Loss: 1.50 Valid F1: 0.688
Epoch: 4 (1.9s) Train Loss: 0.08    Valid Loss: 1.71 Valid F1: 0.644
Epoch: 5 (2.2s) Train Loss: 0.07    Valid Loss: 1.53 Valid F1: 0.690
Epoch: 6 (2.5s) Train Loss: 0.06    Valid Loss: 1.56 Valid F1: 0.700
Epoch: 7 (2.8s) Train Loss: 0.06    Valid Loss: 1.48 Valid F1: 0.680
Epoch: 8 (3.1s) Train Loss: 0.07    Valid Loss: 1.83 Valid F1: 0.606
Epoch: 9 (3.4s) Train Loss: 0.05    Valid Loss: 1.62 Valid F1: 0.624
Epoch: 10 (3.7s) Train Loss: 0

In [10]:
!python pytorch_ladies.py --cuda 0 --dataset cora

python3: can't open file '/content/fastgcn-pytorch/pytorch_ladies.py': [Errno 2] No such file or directory


In [73]:
import subprocess
import re

# Arrays to store results
f1_scores = []
mem_usage = []
train_times = []

for i in range(10):

    print(f"===== Run {i+1} =====")

    # Run the command and capture output
    result = subprocess.run(
        [
            "python3", "fastgcn_test.py",
            "--dataset=Amazon",
            "--early_stop=0",
            "--fast=true",
            "--hidden_dim=16",
            "--norm_feat=false",
            "--lr=0.01",
            "--init_batch=128",
            "--sample_size=64"
        ],
        capture_output=True,
        text=True
    )

    output = result.stdout
    print(output)  # Optional: to see full output each run

    # Extract values with regex
    f1_match = re.search(r"maximum micro F1 testing accuracy:\s*([\d\.]+)\s*%", output)
    mem_match = re.search(r"peak memory usage\s*([\d\.]+)", output)
    time_match = re.search(r"\[TOTAL TIME\]\s*([\d\.]+)", output)

    if f1_match:
        f1_scores.append(float(f1_match.group(1)))
    if mem_match:
        mem_usage.append(float(mem_match.group(1)))
    if time_match:
        train_times.append(float(time_match.group(1)))

# Print results
print("F1 Scores:", f1_scores)
print("Memory Usage (MB):", mem_usage)
print("Training Times (s):", train_times)


===== Run 1 =====
Your model:
FastGCNv2(
  (layers): ModuleList(
    (0-1): 2 x GCNLayer()
  )
  (batch_norms): ModuleList(
    (0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (drop): Dropout(p=0.0, inplace=False)
  (activation): ReLU()
  (final_activation): LogSoftmax(dim=1)
)
========================= STARTING TRAINING =========================
TRAINING INFORMATION:
[DATA] Amazon dataset
[FAST] using FastGCN? True
[INF] using sampling for inference? False
[FEAT] normalized features? False
[DEV] device: cuda:0
[ITERS] performing 200 Adam updates
[LR] Adam learning rate: 0.01
[BATCH] batch size: [128, 64]
RESULTS:
[LOSS] minimum loss: 1.7509845495224
[ACC] maximum micro F1 testing accuracy: 39.63636363636363 %
[BATCH TIME] 0.0053 seconds
[TOTAL TIME] 1.0618 seconds
========================== ENDING TRAINING ==========================

peak memory usage 183.1279296875
Memory after cleanup: 106.68 MB
Figure(600x500)
Figure(600x500)

===== Run 2 

In [74]:
std_acc = np.std(f1_scores)
mean_test_acc = np.mean(f1_scores)
print(f"Test Accuracy: {mean_test_acc:.4f} ± {std_acc:.4f}")
std_training = np.std(train_times)
mean_training = np.mean(train_times)
print(f"training time: {mean_training:.4f} ± {std_training:.4f}")
std_memory = np.std(mem_usage)
mean_memory = np.mean(mem_usage)
print(f"memory used: {mean_memory:.4f} ± {std_memory:.4f}")

Test Accuracy: 37.2727 ± 0.9732
training time: 1.0131 ± 0.0627
memory used: 183.1279 ± 0.0000


In [28]:


import pickle as pkl
import numpy as np
import scipy.sparse as sp
import torch
from torch_geometric.datasets import Amazon
import networkx as nx
import os

# Load Dataset
dataset = Amazon(root='data/amazon', name='Computers')
data = dataset[0]

# Create save directory
save_path = 'planetoid_amazon_computers'
os.makedirs(save_path, exist_ok=True)

# Split manually (you can change this as needed)
def random_split(y, num_classes):
    idx = np.arange(len(y))
    np.random.shuffle(idx)

    train_size = int(0.1 * len(idx))
    val_size = int(0.1 * len(idx))
    test_size = len(idx) - train_size - val_size

    return idx[:train_size], idx[train_size:train_size+val_size], idx[train_size+val_size:]

train_idx, val_idx, test_idx = random_split(data.y.numpy(), dataset.num_classes)

# Save index of test nodes
with open(f"{save_path}/ind.amazon_computers.test.index", 'w') as f:
    for idx in test_idx:
        f.write(f"{idx}\n")

# Convert to scipy sparse matrices
features = sp.csr_matrix(data.x.numpy())
labels = data.y.numpy()

# Features
x = features[train_idx]
tx = features[test_idx]
allx = features

# Labels
y = labels[train_idx]
ty = labels[test_idx]
ally = labels

# Graph (adjacency list)
edge_index = data.edge_index.numpy()
graph = {}
for i in range(data.num_nodes):
    graph[i] = []
for src, dst in edge_index.T:
    graph[src].append(dst)

# Save everything
def save_obj(obj, name):
    with open(os.path.join(save_path, name), 'wb') as f:
        pkl.dump(obj, f)

save_obj(x, 'ind.amazon_computers.x')
save_obj(tx, 'ind.amazon_computers.tx')
save_obj(allx, 'ind.amazon_computers.allx')

save_obj(y, 'ind.amazon_computers.y')
save_obj(ty, 'ind.amazon_computers.ty')
save_obj(ally, 'ind.amazon_computers.ally')

save_obj(graph, 'ind.amazon_computers.graph')


Processing...
Done!
